In [20]:
# Render our plots inline
%matplotlib inline
%pylab inline  
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# General Plotting Parameters
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['font.size'] = 14.
mpl.rcParams['legend.fontsize'] = 12.
mpl.rcParams['axes.labelsize'] = 12.
mpl.rcParams['xtick.labelsize'] = 10.
mpl.rcParams['ytick.labelsize'] = 10.
mpl.rcParams['xtick.minor.pad'] = 4
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'

#Git says this is patched, but it doesn't work from Pip --upgrade 26-mar-2015
#mpl.rcParams['xtick.minor.visible'] = True  

# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
             (255, 187, 120), (44, 160, 44), (152, 223, 138),
              (148, 103, 189),
             (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127),
             (199, 199, 199), (188, 189, 34), (219, 219, 141),
             (23, 190, 207), (158, 218, 229),(214, 39, 40), (255, 152, 150)]  
    
# Scale the RGB values to the [0, 1] range,
# which is the format matplotlib accepts.  
for i in range(len(tableau20)): 
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

Populating the interactive namespace from numpy and matplotlib


In [21]:
import calc_enrich
reload(calc_enrich)
from calc_enrich import calc_del_U
from calc_enrich import stages_per_cascade
from calc_enrich import N_product_by_alpha
from calc_enrich import N_waste_by_alpha
from calc_enrich import machines_per_enr_stage
from calc_enrich import machines_per_strip_stage
from calc_enrich import product_per_enr_stage
from calc_enrich import waste_per_strip_stage
from calc_enrich import Npc_from_Nstages
from calc_enrich import Nwc_from_Nstages
from calc_enrich import machines_per_cascade
from calc_enrich import allowed_feed_per_stage
from calc_enrich import calc_feed_flows

In [22]:
# centrifuge params 
#omega = 64000
#d = 0.1  # m 
#Z = 2.0   # m
#F_m_hrs = 70 # grams/hr  
#T = 320.0   # K
#cut = 0.5
eff = 1.0 # centrifuge efficiency

d = 0.15  # m 
Z = 1.0   # m
F_m_hrs = 15*60*60/(1e3) # grams/hr  
T = 320.0# K
cut = 0.5
omega = 485/(d/2)

# cascade params
Nfc = 0.007
Npc = 0.035
Nwc = 0.003
Fc_month = 739*10000 #kg/month

# The feed enrichment to be used when using a cascade designed for LEU to produce HEU.
Nfc_mod = 0.035
Fc_month_mod = Fc_month #kg/month
#Fc_month_mod = 46 #kg/month

#unit conversions
kg_sec2kg_month = 30.4*24*60*60
v_a = omega * (d/2.0)
F_m = F_m_hrs/(60*60*1000.0)
Fc = Fc_month/kg_sec2kg_month
Fc_mod = Fc_month_mod/kg_sec2kg_month

# Comparison numbers (not needed for calculations)
del_U_th_yr = 1.1 #swu/yr
del_U_obs_yr = 0.71 #Swu/yr
del_U_th = del_U_th_yr/(365.25*24*60*60) #kgSWU/sec
del_U_obs = del_U_obs_yr/(365.25*24*60*60)

Pc_month = 77 #kg/month
Pc = Pc_month/kg_sec2kg_month

qty_len = 6
assay_len = 4

In [23]:
alpha, del_U, del_U_yr = calc_del_U(v_a, Z, d, F_m, T, cut, eff)
ideal_enrich_s, ideal_strip_s= stages_per_cascade(alpha, Nfc, Npc, Nwc)
print "del_U is", del_U, " alpha is ", alpha
print "desired Feed, product, waste", Nfc, Npc, Nwc

int_enrich_stage = int(ideal_enrich_s)+2
int_strip_stage = int(ideal_strip_s)
#feed_flows = calc_feed_flows(int(round(ideal_enrich_s)), int(round(ideal_strip_s)), Fc, cut)
feed_flows = calc_feed_flows(int_enrich_stage, int_strip_stage, Fc, cut)
print ideal_enrich_s,ideal_strip_s

print int_enrich_stage, int_strip_stage
feed_flows


del_U is 3.1939942861e-07  alpha is  1.34782845204
desired Feed, product, waste 0.007 0.035 0.003
4.70933434436 2.44752251091
6 2


array([  3.75142138,   7.50284276,  11.25426413,   9.37855344,
         7.50284276,   5.62713207,   3.75142138,   1.87571069])

In [24]:

Nfs = Nfc
print "Stage   #Mach\t\t Feed\t    Product\t Waste\t   F_assay \tP_assay W_assay"
all_stages = []
for stage_idx in range(int_enrich_stage):
    curr_stage = stage_idx + int_strip_stage
    Fs = feed_flows[curr_stage]
    n_mach_enr = round(machines_per_enr_stage(alpha, del_U, Fs))
    if (n_mach_enr <= 1):
        int_mach = False
    Nps = N_product_by_alpha(alpha, Nfs)
    Ps = product_per_enr_stage(alpha, Nfs, Nps, Fs)
    Ws = Fs - Ps
    Nws = N_waste_by_alpha(alpha, Nfs)
    all_stages.append([stage_idx, n_mach_enr, round(Fs, qty_len), 
                          round(Ps, qty_len), round(Ws, qty_len), round(Nfs, assay_len), 
                          round(Nps, assay_len), round(Nws, assay_len)])
    if (stage_idx == 0):
        W_enr1 = Ws
        Nw_1 = Nws
    print stage_idx, "\t", n_mach_enr,"\t", round(Fs,qty_len), "  ",round(Ps, qty_len), "  ",round(Ws, qty_len),"  ", round(Nfs, assay_len), "\t",round(Nps, assay_len),"\t", round(Nws, assay_len)
    Nfs = Nps

## EQNS SEEM CORRECT BUT ARE UNDER-ESTIMATING BY ONE STAGE. WHY?

Stage   #Mach		 Feed	    Product	 Waste	   F_assay 	P_assay W_assay
0 	2131489.0 	11.254264    5.640833    5.613431    0.007 	0.0094 	0.0052
1 	1776241.0 	9.378553    4.704628    4.673925    0.0094 	0.0126 	0.007
2 	1420993.0 	7.502843    3.76792    3.734923    0.0126 	0.017 	0.0094
3 	1065745.0 	5.627132    2.830171    2.796961    0.017 	0.0227 	0.0126
4 	710496.0 	3.751421    1.890544    1.860878    0.0227 	0.0304 	0.017
5 	355248.0 	1.875711    0.947773    0.927938    0.0304 	0.0405 	0.0227


In [25]:

Nfs = Nw_1
print "Stage   #Mach\t\t Feed\t    Product\t Waste\t   F_assay \tP_assay W_assay"
for stage_idx in range(int_strip_stage-1,-1,-1):
    curr_stage = stage_idx - int_strip_stage
    Fs = feed_flows[stage_idx]
    n_mach_strip = round(machines_per_strip_stage(alpha, del_U, Fs))
    if (n_mach_enr <= 1):
        int_mach = False
    Nps = N_product_by_alpha(alpha, Nfs)
    Nws = N_waste_by_alpha(alpha, Nfs)
    Ws = waste_per_strip_stage(alpha, Nfs, Nws, Fs)
    Ps = Fs - Ws
    all_stages.insert(0,[(curr_stage), n_mach_strip, round(Fs, qty_len), 
                            round(Ps, qty_len), round(Ws, qty_len), round(Nfs, assay_len), 
                            round(Nps, assay_len), round(Nws, assay_len)])
    print (curr_stage), "\t",n_mach_strip ,"\t", round(Fs,qty_len), "  ",round(Ps, qty_len), "  ",round(Ws, qty_len),"  ", round(Nfs, assay_len), "\t",round(Nps, assay_len),"\t", round(Nws, assay_len)
    Nfs = Nws


Stage   #Mach		 Feed	    Product	 Waste	   F_assay 	P_assay W_assay
-1 	1420993.0 	7.502843    2.453359    5.049483    0.0052 	0.007 	0.0039
-2 	710496.0 	3.751421    1.225807    2.525614    0.0039 	0.0052 	0.0029


In [26]:
all_stages


[[-2, 710496.0, 3.751421, 1.225807, 2.525614, 0.0039, 0.0052, 0.0029],
 [-1, 1420993.0, 7.502843, 2.453359, 5.049483, 0.0052, 0.007, 0.0039],
 [0, 2131489.0, 11.254264, 5.640833, 5.613431, 0.007, 0.0094, 0.0052],
 [1, 1776241.0, 9.378553, 4.704628, 4.673925, 0.0094, 0.0126, 0.007],
 [2, 1420993.0, 7.502843, 3.76792, 3.734923, 0.0126, 0.017, 0.0094],
 [3, 1065745.0, 5.627132, 2.830171, 2.796961, 0.017, 0.0227, 0.0126],
 [4, 710496.0, 3.751421, 1.890544, 1.860878, 0.0227, 0.0304, 0.017],
 [5, 355248.0, 1.875711, 0.947773, 0.927938, 0.0304, 0.0405, 0.0227]]

In [27]:
final_w = all_stages[0][4]
final_p = all_stages[len(all_stages)-1][3]
final_f = round(Fc,qty_len)


res = final_f - final_p - final_w
final_f, final_p, final_w, res

(2.813566, 0.947773, 2.525614, -0.6598210000000002)